In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    '''mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    #Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )''' 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    #pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [154]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data2') 

# Actions that we try to detect
actions = np.array(['만나다'])

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 30


In [64]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [65]:
cap = cv2.VideoCapture(0)
key=0
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        if key ==1:
            break
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    key=1
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [69]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data2') 

# Actions that we try to detect
actions = np.array(['funny','happy','healthy','meet'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [70]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [71]:
label_map = {label:num for num, label in enumerate(actions)}

In [72]:
label_map

{'funny': 0, 'happy': 1, 'healthy': 2, 'meet': 3}

In [73]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [78]:
np.array(sequences).shape
X = np.array(sequences)
X.shape

(120, 30, 126)

In [79]:
y = to_categorical(labels).astype(int)


In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [81]:
y_test.shape

(24, 4)

In [82]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [83]:
log_dir = os.path.join('Logs')
checkpoint_path ='체크포인트'
checkpoint_dir = os.path.dirname(checkpoint_path)

#tb_callback = TensorBoard(log_dir=log_dir)
keras_callbacks   = [
      EarlyStopping(monitor='loss', patience=50, mode='min', min_delta=0.0001),
      ModelCheckpoint("최고.h5", monitor='loss', save_best_only=True,save_weights_only=True, mode='min'),
      TensorBoard(log_dir=log_dir)
]

In [84]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [85]:
actions[np.argmax(res)]

'funny'

In [86]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [87]:
model.fit(X_train, y_train, epochs=800, callbacks=[keras_callbacks])

Epoch 1/800
3/3 [==============================] - 4s 47ms/step - loss: 1.3800 - categorical_accuracy: 0.1979
Epoch 2/800
3/3 [==============================] - 0s 51ms/step - loss: 1.3600 - categorical_accuracy: 0.2708
Epoch 3/800
3/3 [==============================] - 0s 49ms/step - loss: 1.2957 - categorical_accuracy: 0.2708
Epoch 4/800
3/3 [==============================] - 0s 48ms/step - loss: 1.1299 - categorical_accuracy: 0.3646
Epoch 5/800
3/3 [==============================] - 0s 39ms/step - loss: 1.6001 - categorical_accuracy: 0.5417
Epoch 6/800
3/3 [==============================] - 0s 40ms/step - loss: 1.4224 - categorical_accuracy: 0.2188
Epoch 7/800
3/3 [==============================] - 0s 38ms/step - loss: 1.3920 - categorical_accuracy: 0.2188
Epoch 8/800
3/3 [==============================] - 0s 39ms/step - loss: 1.3783 - categorical_accuracy: 0.4167
Epoch 9/800
3/3 [==============================] - 0s 40ms/step - loss: 1.3534 - categorical_accuracy: 0.4896
Epoch 10/8

3/3 [==============================] - 0s 39ms/step - loss: 0.3677 - categorical_accuracy: 0.9062
Epoch 75/800
3/3 [==============================] - 0s 48ms/step - loss: 0.2936 - categorical_accuracy: 0.9062
Epoch 76/800
3/3 [==============================] - 0s 38ms/step - loss: 0.3123 - categorical_accuracy: 0.8958
Epoch 77/800
3/3 [==============================] - 0s 48ms/step - loss: 0.2579 - categorical_accuracy: 0.8958
Epoch 78/800
3/3 [==============================] - 0s 37ms/step - loss: 0.2886 - categorical_accuracy: 0.8854
Epoch 79/800
3/3 [==============================] - 0s 49ms/step - loss: 0.2515 - categorical_accuracy: 0.8854
Epoch 80/800
3/3 [==============================] - 0s 46ms/step - loss: 0.2190 - categorical_accuracy: 0.9062
Epoch 81/800
3/3 [==============================] - 0s 46ms/step - loss: 0.1995 - categorical_accuracy: 0.9167
Epoch 82/800
3/3 [==============================] - 0s 48ms/step - loss: 0.1979 - categorical_accuracy: 0.9167
Epoch 83/800
3

3/3 [==============================] - 0s 42ms/step - loss: 0.7219 - categorical_accuracy: 0.6979
Epoch 148/800
3/3 [==============================] - 0s 38ms/step - loss: 0.6838 - categorical_accuracy: 0.6979
Epoch 149/800
3/3 [==============================] - 0s 39ms/step - loss: 0.6239 - categorical_accuracy: 0.6875
Epoch 150/800
3/3 [==============================] - 0s 39ms/step - loss: 0.5501 - categorical_accuracy: 0.6875
Epoch 151/800
3/3 [==============================] - 0s 38ms/step - loss: 0.5075 - categorical_accuracy: 0.6979
Epoch 152/800
3/3 [==============================] - 0s 40ms/step - loss: 0.4949 - categorical_accuracy: 0.6979
Epoch 153/800
3/3 [==============================] - 0s 40ms/step - loss: 0.4718 - categorical_accuracy: 0.6667
Epoch 154/800
3/3 [==============================] - 0s 38ms/step - loss: 0.4344 - categorical_accuracy: 0.6875
Epoch 155/800
3/3 [==============================] - 0s 38ms/step - loss: 0.3829 - categorical_accuracy: 0.7083
Epoch 

In [93]:
model.load_weights('최고.h5')

In [89]:
res = model.predict(X_test)

1/1 [==============================] - 0s 488ms/step


In [90]:
actions[np.argmax(res[0])]

'healthy'

In [91]:
actions[np.argmax(y_test[0])]

'healthy'

In [92]:
model.save('심화테스트4.h5')

In [112]:
del model

In [232]:
model.load_weights('심화테스트4.h5')

In [94]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [95]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 30ms/step


In [96]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [97]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[17,  0],
        [ 0,  7]],

       [[20,  0],
        [ 0,  4]],

       [[15,  0],
        [ 2,  7]],

       [[18,  2],
        [ 0,  4]]], dtype=int64)

In [98]:
accuracy_score(ytrue, yhat)


0.9166666666666666

In [99]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), -1)#colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        print(int(prob*100))
    return output_frame

In [102]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.7) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 25ms/step
happy
1
98
0
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
happy
0
99
0
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
happy
0
99
0
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
happy
0
99
0
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
happy
0
99
0
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
happy
0
99
0
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
happy
0
99
0
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
happy
0
99
0
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [===

healthy
0
0
100
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
healthy
0
0
100
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
healthy
0
0
100
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
healthy
0
0
100
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
healthy
0
0
100
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
healthy
0
0
100
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
healthy
0
0
100
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
healthy
0
0
100
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================]

In [ ]:
qqqq